In [1]:
import numpy as np
import yaml

with open("../data/D_germline_ex.yaml", 'r') as stream:
    try:
        y = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Build a dictionary of the YAML keyed by state name.
state_d = {}
for s in y['states']:
    state_d[s['name']] = s

bases = list('ACGT')
insert_names = ['insert_left_'+x for x in bases]
dummy_names = ['dummy_D_'+str(i) for i in range(5)]

In [2]:
def stack_rows(array_list):
    return np.concatenate([np.array([a]) for a in array_list])

def extract_transitions(d, names):
    def get_with_default(n):
        if n in d:
            return d[n]
        else:
            return 0.
    v = np.array([get_with_default(n) for n in names])
    return v

In [3]:
# NTI transition and emission matrices.

in_landing = extract_transitions(state_d['init']['transitions'], insert_names)

T = stack_rows([
        extract_transitions(state_d[n]['transitions'], insert_names) 
        for n in insert_names])

E = stack_rows([
        np.array([state_d[n]['emissions']['probs'][b] for b in bases]) 
        for n in insert_names])

out_landing = stack_rows([
        extract_transitions(state_d[i_n]['transitions'], dummy_names) 
        for i_n in insert_names])

# Original version
left_flex_ind = (3, 5)
right_flex_ind = (6, 9)
# Temporary version working out confusion.
left_flex_ind = (4, 5)
right_flex_ind = (6, 8)
# Relpos for the dummy D gene
right_relpos = 5
emission_indices = [0, 1, 0, 2, 3, 0, 1, 1, 1, 3, 2, 3, 3]
used_emission_indices = emission_indices[left_flex_ind[0]:right_flex_ind[1]]

for o in [in_landing, T, E, out_landing, used_emission_indices]:
    print o

[ 0.1   0.2   0.1   0.05]
[[ 0.075  0.175  0.05   0.025]
 [ 0.075  0.175  0.05   0.025]
 [ 0.075  0.175  0.05   0.025]
 [ 0.075  0.175  0.05   0.025]]
[[ 0.7  0.1  0.1  0.1]
 [ 0.1  0.7  0.1  0.1]
 [ 0.1  0.1  0.7  0.1]
 [ 0.1  0.1  0.1  0.7]]
[[ 0.45   0.125  0.1    0.     0.   ]
 [ 0.45   0.125  0.1    0.     0.   ]
 [ 0.45   0.125  0.1    0.     0.   ]
 [ 0.45   0.125  0.1    0.     0.   ]]
[3, 0, 1, 1]


In [4]:
# Following the image in the issue literally:
all_A = []
for t in range(len(used_emission_indices)):
    y_t = used_emission_indices[t]
    A = np.zeros((t+1, 4))
    if t > 0:
        A[:t, :] = all_A[t-1].dot(T) * E[y_t, :]
    A[t, :] = in_landing * E[y_t, :]
    all_A.append(A)

However, we don't need all of the contents of the As.
We only need to start at positions 3 and 4, which because of the setup of used_emission_indices are the first two entries
(i.e. the left flex.)

In [5]:
left_flex = left_flex_ind[1]-left_flex_ind[0]
sub_A = [A[:(left_flex+1), :] for A in all_A]
sub_A

[array([[ 0.01 ,  0.02 ,  0.01 ,  0.035]]),
 array([[ 0.0039375,  0.0013125,  0.000375 ,  0.0001875],
        [ 0.07     ,  0.02     ,  0.01     ,  0.005    ]]),
 array([[  4.35937500e-05,   7.12031250e-04,   2.90625000e-05,
           1.45312500e-05],
        [  7.87500000e-04,   1.28625000e-02,   5.25000000e-04,
           2.62500000e-04]]),
 array([[  5.99414063e-06,   9.79042969e-05,   3.99609375e-06,
           1.99804688e-06],
        [  1.08281250e-04,   1.76859375e-03,   7.21875000e-05,
           3.60937500e-05]])]

We want the matrix that has been run out to one position before the LHS of the right flex (but we subtract off `left_flex_ind[0]` because we're starting the computation at that position; see definition of `used_emission_indices`).
We then right multiply by the column vector that corresponds to the entrypoint of the D gene. 

In [6]:
for i in range(0,right_flex_ind[1]-right_flex_ind[0]+1):
    print sub_A[i+right_flex_ind[0]-left_flex_ind[0]-1].dot(out_landing[:,i+right_flex_ind[0]-right_relpos])

[ 0.00072656  0.013125  ]
[  7.99218750e-05   1.44375000e-03]
[ 0.  0.]
